In [17]:
from cf_units import Unit
from IPython.core.display import clear_output
import iris
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnchoredText
import matplotlib.patheffects as PathEffects
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
import pandas as pd
from pathlib import Path
import xarray as xr
from tqdm import tqdm_notebook as tqdm
import string

import arke
from arke.cart import lcc_map, lcc_map_grid

from common_defs import winters, nyr, winter_dates, toponyms, aliases, dset_names
from plot_utils import LCC_KW, trans, clev101, abs_plt_kw, iletters
import mypaths

from octant.core import TrackRun, OctantTrack, HOUR
from octant.misc import calc_all_dens, SUBSETS, DENSITY_TYPES
import octant
octant.__version__

'0.0.11'

In [2]:
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning, module='dask')
warnings.filterwarnings('ignore', category=UserWarning, module='iris')
warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

In [3]:
plt.style.use('paperfig.mplstyle')

In [4]:
lsm = xr.open_dataarray(mypaths.era5_dir / 'lsm.nc').squeeze()
lsm.attrs['units'] = 1
lon2d, lat2d = np.meshgrid(lsm.longitude, lsm.latitude)

#### Grids and arrays for density calculation

In [5]:
lon_dens1d = np.arange(-20., 50.1, 1) # 0.3)
lat_dens1d = np.arange(65., 85.1, 1) # 0.3)
lon_dens, lat_dens = np.meshgrid(lon_dens1d, lat_dens1d)

In [6]:
lsm_1deg = lsm.interp(coords=dict(longitude=lon_dens[0, :], latitude=lat_dens[:, 0])).to_iris()
lsm_1deg.coord('longitude').units = Unit('degrees_east')
lsm_1deg.coord('latitude').units = Unit('degrees_north')
lsm_1deg.coord('longitude').guess_bounds()
lsm_1deg.coord('latitude').guess_bounds()

In [7]:
weights = lsm_1deg.copy(data=iris.analysis.cartography.area_weights(lsm_1deg, normalize=False))
weights.units = Unit('m^2')
weights.rename('area_weights')
weights.convert_units('km^2')

In [8]:
area_weights_1deg = xr.DataArray.from_iris(weights)

In [9]:
area_weights_1deg_norm = xr.DataArray.from_iris(lsm_1deg.copy(data=iris.analysis.cartography.area_weights(lsm_1deg, normalize=True)))

In [10]:
lsm_1deg = xr.DataArray.from_iris(lsm_1deg)

### Mean sea ice edge position

In [11]:
sea_ice_ds = xr.open_mfdataset(sorted(mypaths.era5_dir.glob('*.ci.nc')))

In [12]:
sea_ice_conc = sea_ice_ds.ci[:, (sea_ice_ds.latitude >= 65) & (sea_ice_ds.latitude <= 85), (sea_ice_ds.longitude >= -20) & (sea_ice_ds.longitude <= 50)]

In [13]:
sic_thresh = 0.15  # 15% threshold

In [14]:
sea_ice_conc_mean = sea_ice_conc.mean(dim='time')

## Calculate density

In [18]:
r = 111.3
grid_str = r'$1^\degree\times 1^\degree$'

In [19]:
# for (dset_name, _) in tqdm(dset_names):
    
#     all_dens = calc_all_dens(track_runs[dset_name], lon_dens, lat_dens, method='radius', r=r)
#     attrs = all_dens.attrs.copy()
#     all_dens = all_dens / nyr
#     all_dens.attrs.update(attrs)
    
#     all_dens.to_netcdf(mypaths.procdir / f'{dset_name}_2008_2017_top10_all_densities_r{round(r):3d}.nc')
    
# clear_output()

In [20]:
# all_dens.to_netcdf(mypaths.procdir / f'{dataset}_run{run_id_start+run_id:03d}_2008_2017_all_densities_r{round(r):3d}.nc')

In [21]:
AXGR_KW = dict(axes_pad=0.45,
               cbar_location='right',
               cbar_mode='single',
               cbar_pad=0.1,
               cbar_size='3%')
diff_plt_kw = dict(cmap='coolwarm', extend='both', **trans)
cntr_kw = dict(colors='#222222', linewidths=0.5, **trans)
cntr_lab_kw = dict(fmt='%3.0f', colors='k')
ci_kw = dict(levels=[0.15], linewidths=4, **trans)
at_kw = dict(loc=2, prop=dict(size='small'))
text_kw = dict(ha='center',
               fontsize='xx-large',
               path_effects=[PathEffects.withStroke(linewidth=3,
                                                    foreground='w')])

In [ ]:
from ipywidgets import interact
@interact(dens_type=DENSITY_TYPES, subset=SUBSETS, dset_name=[i[0] for i in dset_names])
def fun(dset_name, dens_type='track', subset='moderate'):
    
    all_dens = xr.open_dataarray(mypaths.procdir / f'{dset_name}_2008_2017_top10_all_densities_r{round(r):3d}.nc')
    
    fig = plt.figure(figsize=(10, 10))
    ax = lcc_map(fig, **LCC_KW)

    h = all_dens.sel(subset=subset, dens_type=dens_type).plot.contourf(add_colorbar=False, ax=ax, **abs_plt_kw)
    cb = fig.colorbar(h, pad=0.01, shrink=0.7)
#     ax.plot(13, 74, marker='o', **mapkey)

# ttl_str = "\n".join([f"{k} = {v}" for k, v in density_kw.items()])
# ax.add_artist(AnchoredText(f'{dens2show.capitalize()} density (per year)\n{dataset}\n{ttl_str}', loc=2))

# for _, tr in TR[subset].groupby('track_idx'):
#     tr.plot_track(ax=ax);

In [ ]:
# fig.savefig(mypaths.plotdir / 'climatology' / f'pmctrack_density_point_{dataset}_{density_kw["subset"]}_r{density_kw["r"]:3.0f}.{fmt}', **svfigkw)

In [22]:
subsets = SUBSETS[1:]

In [23]:
# AXGR_KW = dict(axes_pad=0.4,
#                cbar_location='right',
#                cbar_mode='each',
#                cbar_pad=0.05,
#                cbar_size='3%')
AXGR_KW = dict(axes_pad=0.3)
diff_plt_kw = dict(cmap='coolwarm', extend='both', **trans)
cntr_kw = dict(colors='#222222', linewidths=0.5, **trans)
cntr_lab_kw = dict(fmt='%3.0f', colors='k')
ci_kw = dict(levels=[0.15], linewidths=2, **trans)
at_kw = dict(loc='upper right', prop=dict(size='small'))
# text_kw = dict(ha='center',
#                fontsize='xx-large',
#                path_effects=[PathEffects.withStroke(linewidth=3,
#                                                     foreground='w')])

In [24]:
ncol = len(dset_names)
nrow = len(subsets)

for dens_type in tqdm(DENSITY_TYPES, desc='figures', leave=False):
    fig = plt.figure(figsize=(ncol*5, nrow*5))
    axgr = lcc_map_grid(fig, (nrow, ncol), **LCC_KW, **AXGR_KW)

    ttl = f'{dens_type.capitalize()} density\nr = {r} km, {grid_str}\n2008-2017 (9 winters)'
    fig.suptitle(ttl, x=axgr.axes_all[0].get_position().get_points()[0, 0],
                 transform=axgr.axes_all[0].transAxes, ha='left', fontsize='large')

    iletters = iter(string.ascii_lowercase)
    for ax in axgr.axes_all:
        ax.set_title(f'({next(iletters)})', loc='left', fontsize='medium')
    # iter_cax = iter(axgr.cbar_axes)
    for axcol, (dset_name, dset_label) in tqdm(zip(axgr.axes_column, dset_names), desc='datasets', leave=False):
        
        all_dens = xr.open_dataarray(mypaths.procdir / f'{dset_name}_2008_2017_top10_all_densities_r{round(r):3d}.nc')
        for ax, subset in tqdm(zip(axcol, subsets), desc='subsets', leave=False):
            
            data = all_dens.sel(subset=subset, dens_type=dens_type)
            lab = "\n".join(dset_label.split(", "))
            txt = f'{lab}\n{aliases[subset]}'
            ax.add_artist(AnchoredText(txt, **at_kw))
#             try:
#                 h = data.plot.contourf(ax=ax, robust=True, add_colorbar=False, add_labels=False, **abs_plt_kw)
#             except:
            h = data.plot.contourf(ax=ax, robust=False, add_colorbar=False, add_labels=False, **abs_plt_kw)

#             # levels = h.get_array()
#             hh = ax.contour(lon_dens, lat_dens, ma_data, **cntr_kw)
#             hhh = ax.clabel(hh, **cntr_lab_kw)
#             plt.setp(hhh, path_effects=[PathEffects.withStroke(linewidth=1.5, foreground='w')])
            # Overlay with sea ice edge
            sea_ice_conc_mean.plot.contour(ax=ax, add_labels=False, colors='C0', **ci_kw)
        
            cax = inset_axes(ax, borderpad=0.5,
                     width="4%",
                     height="45%",
                     loc='upper left')
            
            # cax = next(iter_cax)
            cb = fig.colorbar(h, orientation='vertical', cax=cax)
            cb.ax.tick_params(labelsize='large')
            for i in cb.ax.get_yticklabels():
                i.set_path_effects([PathEffects.withStroke(linewidth=2, foreground='w')])

    fig.savefig(mypaths.plotdir / f'pmctrack_era5_vs_interim_{dens_type}_density_r{round(r):3d}')
    plt.close()

## Another option for counting....

### Load tracks

In [ ]:
track_runs = dict()
for (dset_name, _) in tqdm(dset_names):
    TR = TrackRun()
    TR.data = OctantTrack.from_mux_df(pd.read_parquet(mypaths.procdir / f'{dset_name}_2008_2017_top10.parquet', engine='pyarrow'))
    TR.is_categorised = True
    track_runs[dset_name] = TR
clear_output()

In [ ]:
from scipy.ndimage.filters import gaussian_filter

In [ ]:
dens = track_runs['era5_run000'].density(lon_dens, lat_dens, by='track',
                  method='cell', r=r, subset='strong')

In [ ]:
a = dens * (area_weights_1deg/area_weights_1deg.max())

In [ ]:
aa = xr.apply_ufunc(gaussian_filter, a, kwargs=dict(sigma=1))

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = lcc_map(fig, **LCC_KW)

h = aa.plot.contourf(add_colorbar=False, ax=ax, levels=np.arange(0, 50, 5)*0.2, **abs_plt_kw)
cb = fig.colorbar(h, pad=0.01, shrink=0.7)